In [31]:
from PIL import Image
from pathlib import Path
import argparse
import glob
import os
import pandas as pd
from re import search
import re


In [3]:
path = '/home/ubuntu/bucket/projects'
projdir = '2019_07_11_JUMP-CP-pilots'
batchname = '2020_11_06_Scope1_MolDev'



In [4]:

    
plist = [f for f in os.listdir(os.path.join(path, projdir, batchname,'images'))]

platelist = []
flist = []

for p in plist:


    pl = p.split("-")
    
    plate = '_'.join(pl)

    outpath = os.path.join(path, projdir, 'workspace', 'load_data_csv', batchname, plate)
    
    platelist.append(plate)
    
    

    def makedirectory():

        try:
            os.makedirs(outpath)

            print("Directory", outpath, "is created")

        except IOError:

            print("Directory", outpath, "already exists")

    directory = makedirectory()


    fpath = os.path.join(path, projdir, batchname,'images', p, 'TimePoint_1')
    
    flist.append(fpath)

#     df = load_data()

#     df.to_csv(outpath + "/" + 'load_data.csv', index=False)

#     df_with_illum = load_data_with_illum()

#     df_with_illum.to_csv(outpath + "/" + 'load_data_with_illum.csv', index=False)






    



In [4]:
channels = {'ch1':"OrigMito",
    'ch2':"OrigActin",
    'ch3':"OrigGolgi",
    'ch4':"OrigRNA",
    'ch5':"OrigER",
    'ch6':"OrigDNA",
    }
## for Adaptive folder

# channels = {'ch1':"OrigDNA",
#     'ch2':"OrigMito",
#     'ch3':"OrigActin",
#     'ch4':"OrigGolgi",
#     'ch5':"OrigRNA",
#     'ch6':"OrigER",
#     }

In [86]:
def load_data(channels):
    
    lst = []

    for i, image in enumerate(os.listdir(fpath)):
    
        imgpath = os.path.join(fpath, image)


        if imgpath.endswith("TIF"):

                head, tail = os.path.split(imgpath)

                lst.append(tail)

        keys = list(channels.keys())
        values = list(channels.values())


        ch1 = [s for s in lst if 'w1' in s]
        ch2 = [s for s in lst if 'w2' in s]
        ch3 = [s for s in lst if 'w3' in s]
        ch4 = [s for s in lst if 'w4' in s]
        ch5 = [s for s in lst if 'w5' in s]
        ch6 = [s for s in lst if 'w6' in s]


        zippedlist = list(zip(ch1, ch2, ch3, ch4, ch5, ch6))


        df = pd.DataFrame(zippedlist, columns=["FileName_"+chname for chname in values])

        path_columns = { c : fpath for c in ["PathName_"+chname for chname in values]}

        df = df.assign(**path_columns)


        wellname = df['FileName_OrigDNA'].tolist()
        


        pattern = re.compile("Plate3-PCO-6ch-Adaptive-20XPA_(?P<Well>\w+)_s(?P<site>\d+)")


        match = [pattern.match(i) for i in wellname]
        well = [r.group("Well") for r in match]
        site= [f.group("site") for f in match]

        df['Metadata_Well'] = well
        df['Metadata_Site'] = site
        df['Metadata_Plate'] = plate


        colnames = ['FileName_'+values[0], 'PathName_'+values[0],
                    'FileName_'+values[1], 'PathName_'+values[1],
                    'FileName_'+values[2], 'PathName_'+values[2],
                    'FileName_'+values[3], 'PathName_'+values[3],
                    'FileName_'+values[4], 'PathName_'+values[4],
                    'FileName_'+values[5], 'PathName_'+values[5],
                    'Metadata_Plate','Metadata_Well', 
                    'Metadata_Site']

   
    
    return df.reindex(columns=colnames)

df = load_data(channels) 

df.to_csv(outpath + "/" + 'load_data.csv', index=False)

In [227]:
path = '/Users/habbasi/Desktop/2020_11_24_Scope1_Nikon/BR00117063b10x/load_data.csv'
outpath = '/Users/habbasi/Desktop/2020_11_24_Scope1_Nikon/BR00117063b10x/'

batchpath = '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_24_Scope1_Nikon'

df = pd.read_csv(path)

new = os.path.split(df['PathName_OrigAGP'][0])[-2]

print(new)



plates = 'BR00117063b10x'



/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117063b10x__20201118_114204_306


In [228]:

recolumns = ['Group_Index', 'PathName_OrigAGP', 'PathName_OrigDNA',
       'PathName_OrigER', 'PathName_OrigMito', 'FileName_OrigAGP',
       'FileName_OrigDNA', 'FileName_OrigER', 'FileName_OrigMito',
       'Series_OrigAGP', 'Series_OrigDNA', 'Series_OrigER', 'Series_OrigMito',
       'Frame_OrigAGP', 'Frame_OrigDNA', 'Frame_OrigER', 'Frame_OrigMito',
       'Channel_OrigAGP', 'Channel_OrigDNA', 'Channel_OrigER',
       'Channel_OrigMito', 'Metadata_ColorFormat', 'Metadata_Plate',
       'Metadata_Series', 'Metadata_SizeC', 'Metadata_Site', 'Metadata_SizeX',
       'Metadata_SizeY', 'Metadata_SizeZ', 'Metadata_T', 'Metadata_Well',
       'Metadata_Z']

drop_columns= ['PathName_OrigAGP', 'PathName_OrigDNA',
       'PathName_OrigER', 'PathName_OrigMito']

df = df.drop(drop_columns, axis=1)



df['PathName_OrigAGP']= new
df['PathName_OrigDNA']= new
df['PathName_OrigMito']= new
df['PathName_OrigER']= new



dp = df.to_csv(outpath + "/" + 'load_data.csv', index=False)


In [230]:
for i in range(0,9):
    print(i)

0
1
2
3
4
5
6
7
8


In [229]:
df['FileName_IllumDNA'] = plates + '_IllumDNA.npy'

df['PathName_IllumDNA'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumER'] = plates+'_IllumER.npy'

df['PathName_IllumER'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumAGP'] = plates+'_IllumAGP.npy'

df['PathName_IllumAGP'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumMito'] = plates+'_IllumMito.npy'

df['PathName_IllumMito'] = os.path.join(batchpath, 'illum', plates)


df.to_csv(outpath + "/" + 'load_data_with_illum.csv', index=False)




'/Users/habbasi/Desktop/JUMP/2020_11_24_Scope1_Nikon/BR00117061a'

In [58]:
df['FileName_IllumDNA'] = plates+'_IllumDNA.npy'

df['PathName_IllumDNA'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumER'] = plates+'_IllumER.npy'

df['PathName_IllumER'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumAGP'] = plates+'_IllumAGP.npy'

df['PathName_IllumAGP'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumMito'] = plates+'_IllumMito.npy'

df['PathName_IllumMito'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumRNA'] = plates+'_IllumRNA.npy'

df['PathName_IllumRNA'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumBrightField'] = plates+'_IllumBrightField.npy'

df['PathName_IllumBrightField'] = os.path.join(batchpath, 'illum', plates)


df.to_csv(outpath + "/" + 'load_data_with_illum.csv', index=False)



In [356]:
path = '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS'



batchpath='/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_16_Scope1_YokogawaUS'

lstdir = sorted([os.path.join(path, f) for f in os.listdir(path)])
lstdir

['/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/.DS_Store',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117014_3D_2x2x1_singleplane_10x_20201026_113115',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117014_3D_2x2x1_singleplane_20x_20201026_113115',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117033_3D_2x2x1_dpc_20x_20201023_151718',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117033_3D_2x2x1_singleplane_20x_20201015_161709',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117056_2x2x1_20x_20201022_110522',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117056_3D_2x2x1_20x_20201022_173956',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117059_20X_20201010_140319',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO0117059_40x_20201013_155609',
 '/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO01177034_20x_20201020_164856']

In [357]:

ppath = lstdir[9]

ppath

'/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS/BRO01177034_20x_20201020_164856'

In [358]:



df = pd.read_csv(os.path.join(ppath, 'load_data.csv'))

p1 = df.Metadata_Plate[0].split("_")[0]

p2 = df.Metadata_Plate[0].split("_")[1]
plates = p1 + "_" + p2 



plates

'BRO01177034_20x'

In [359]:
#df = df.drop(["Metadata_Plate", "Metadata_Zplane"], axis=1)
df = df.drop(["Metadata_Plate"], axis=1)

df["Metadata_Plate"] = str(plates)
outpath = os.path.join('/Users/habbasi/Desktop/JUMP/2020_11_16_Scope1_YokogawaUS',plates)
os.makedirs(outpath)
dp = df.to_csv(outpath + "/" + 'load_data.csv', index=False)

In [360]:
df['FileName_IllumDNA'] = plates+'_IllumDNA.npy'

df['PathName_IllumDNA'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumER'] = plates+'_IllumER.npy'

df['PathName_IllumER'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumAGP'] = plates+'_IllumAGP.npy'

df['PathName_IllumAGP'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumMito'] = plates+'_IllumMito.npy'

df['PathName_IllumMito'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumRNA'] = plates+'_IllumRNA.npy'

df['PathName_IllumRNA'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumBrightField'] = plates+'_IllumBrightField.npy'

df['PathName_IllumBrightField'] = os.path.join(batchpath, 'illum', plates)


df.to_csv(outpath + "/" + 'load_data_with_illum.csv', index=False)




In [364]:
sorted(os.listdir("/Users/habbasi/Desktop/2020_11_16_Scope1_YokogawaUS"))

['BRO0117014_10x',
 'BRO0117014_20x',
 'BRO0117033_20x',
 'BRO0117033_20xb',
 'BRO0117056_20x',
 'BRO0117056_20xb',
 'BRO0117059_20X',
 'BRO0117059_40x',
 'BRO01177034_20x']

In [141]:
df = pd.read_csv(os.path.join(ppath, 'load_data.csv'))


df['Metadata_Plate'] = df['Metadata_Plate'].str.replace("Plate3_PCO_6ch_Adaptive_20XPA_Plate_20118", "Plate3_PCO_6ch_Adaptive_20XPA")

plates = df.Metadata_Plate[0]
plates

'Plate3_PCO_6ch_Adaptive_20XPA'

In [361]:
for s in range(1,2):
    print(s)

1


In [142]:
from PIL import Image
from pathlib import Path
import argparse
import glob
import os
import pandas as pd
from re import search
import re 



outpath = "/Users/habbasi/Desktop/2020_11_06_Scope1_MolDev/" + plates

os.makedirs(outpath)

dp = df.to_csv(outpath + "/" + 'load_data.csv', index=False)

In [143]:
batchpath = '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP-pilots/2020_11_06_Scope1_MolDev'


# channels = {'ch1':"OrigMito",
#     'ch2':"OrigActin",
#     'ch3':"OrigGolgi",
#     'ch4':"OrigRNA",
#     'ch5':"OrigER",
#     'ch6':"OrigDNA",
#     }

channels = {'ch1':"OrigDNA",
    'ch2':"OrigMito",
    'ch3':"OrigActin",
    'ch4':"OrigGolgi",
    'ch5':"OrigRNA",
    'ch6':"OrigER",
    }

In [144]:
df['FileName_IllumDNA'] = plates+'_IllumDNA.npy'

df['PathName_IllumDNA'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumMito'] = plates+'_IllumMito.npy'

df['PathName_IllumMito'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumMito'] = plates+'_IllumMito.npy'

df['PathName_IllumMito'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumActin'] = plates+'_IllumActin.npy'

df['PathName_IllumActin'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumGolgi'] = plates+'_IllumGolgi.npy'

df['PathName_IllumGolgi'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumRNA'] = plates+'_IllumRNA.npy'

df['PathName_IllumRNA'] = os.path.join(batchpath, 'illum', plates)

df['FileName_IllumER'] = plates+'_IllumER.npy'

df['PathName_IllumER'] = os.path.join(batchpath, 'illum', plates)




df.to_csv(outpath + "/" + 'load_data_with_illum.csv', index=False)